In [172]:
# Importing Libraries
from ipywidgets import VBox, Label, interactive_output, Output
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import math

In [173]:
# ---------------------------
# ----- Widget Creation -----
# ---------------------------

# duration
duration = widgets.BoundedIntText(
    value=12,
    min=1,
    max=24,
    step=1,
    description='Months:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '100px'},
    disabled=False
)

# inference rate
inference_rate = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 800,
    step = 1,
    description = 'Inferences/Hour: ',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '100px'},
    disabled = False
)

# battery type
battery_type = widgets.Dropdown(
    options=[('Li-Po', 0), ('Li-Ion', 1)],
    value= 0,
    description='Battery Type:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '100px'},
)

# Build vertical layout to look like the screenshot
ui = VBox([
    Label("Please enter a deployment duration:"),
    duration,
    Label("Please enter an inference rate:"),
    inference_rate,
    Label("Please enter a battery type:"),
    battery_type
])

In [174]:
# -------------------------------
# ----- Dynamic Update Logic ----
# -------------------------------

def update_all_outputs(duration_val, inference_rate_val, battery_type_val):
    with output:
        output.clear_output(wait=True)
        # ---- Ah calculations ----

        # Baseline: 12.4 mA
        # Mic: 10.2 mA
        # Inference & mel spec: 2.6 mA
        # SD write: 3 mA
        # Piezo: 50 nA

        # Active current = baseline + inference & mel spec + mic + SD write + piezo = ~28.20005 mA
        active_current = 28.20005 / 1000  # store in amps

        # Idle current = standby mode current consumption = ~2.9 mA
        idle_current = 2.9 / 1000  # store in amps

        # Convert duration from months to hours -> T_hours = T_months * 30.1467 days/month * 24 hr/day
        duration_hours = duration_val * 30.1467 * 24

        # Fractional representation of inference activations in an hour -> Inference Rate * (4 sec inference)/(3600 sec/hr)
        active_inf = inference_rate_val * (4 / 3600)

        # Amp Hours = T_hours * [(A_active * (I_rate * 4/3600) + (A_idle * (1 - (I_rate * 4/3600)))]
        charge = duration_hours * ((active_current * active_inf) + idle_current * (1 - active_inf))

        # ---- Battery Weight Calculations ----

        # Energy (Wh) = charge (Ah) * Nominal Voltage (V)
        energy = charge * 3.3

        # Set energy density to average weight (kg) for selected battery type
        energy_density = 250 if battery_type_val else 150

        # Weight (lbs) = Energy (Wh) / Energy Density (Wh/kg) * 2.205 (conversion factor)
        weight = (energy / energy_density) * 2.205  # Convert to lbs

        # ---- SD Card Size Calculations ----

        # Sample Rate = 16000 Hz
        # Channels = 1
        # Bit depth = 16 bits
        # Inference duration = 4 seconds

        # File size (Bytes) = (Sample Rate (Hz) * Duration (sec) * Channels * Bit Depth (bits/channel)) / Byte Conv (bits/byte)
        file_size = (16000 * 4 * 1 * 16) / 8

        # SD size (Bytes) = File size (Bytes) * Inference rate (Inf/hr) * Duration (hr)
        sd_size = (file_size * inference_rate_val * duration_hours) / 1e9  # in GB

        # ---- Fermi Estimations Output ----

        print("\n========Selections========")
        print(f"Inference Rate: {inference_rate_val} (inferences/hour)")
        print(f"Duration: {duration_val} (months)")
        print("==========================")

        print("\n========Fermi Estimations========")
        print("Current Measurements (mA):")
        print("Baseline = 12.4 mA")
        print(f"Mic = {10.2} mA")
        print(f"Inference & Mel Spec = {2.6} mA")
        print(f"SD Write = {3} mA")
        print(f"Piezo = {50} nA")
        print(f"Idle Current = {idle_current * 1000:.2f} mA\n")

        print("Active Current Calculations (mA):")
        print("Active Current = Baseline + Inference & Mel Spec + Mic + SD Write + Piezo")
        print(f"Active Current = {active_current * 1000:.2f} mA\n")

        print("Charge (Ah) Calculations:")
        print("Duration (hr) = Duration (months) * 30.1467 days/month * 24 hr/day")
        print(f"Duration (hr) = {duration_hours:.2f} hr")
        print(f"Charge = Duration (hr) * [(Active Current * 4/3600) + (Idle Current * (1 - 4/3600))]")
        print(f"Charge = {duration_hours:.2f} * [({active_current:.5f} * {active_inf:.5f}) + ({idle_current:.5f} * (1 - {active_inf:.5f}))]")
        print(f"Charge: {charge:.2f} Ah\n")

        print("Weight (lbs) Calculations:")
        print("Energy (Wh) = Charge (Ah) * Nominal Voltage (V)")
        print(f"Energy (Wh) = {charge:.2f} Ah * 3.3 V")
        print("Weight (lbs) = Energy (Wh) / Energy Density (Wh/kg) * 2.205")
        print(f"Weight: {weight:.2f} lbs\n")

        print("SD Card Size Calculations (GB):")
        print("File Size (Bytes) = (16000 * 4 * 1 * 16) / 8")
        print(f"File Size (Bytes) = {file_size} Bytes")
        print("SD Size (GB) = File Size * Inference Rate * Duration / 1e9")
        print(f"SD Size: {sd_size:.2f} GB")
        print("=================================\n")

        # ---- Plot Helper ----

        def plot_with_top_limit(title, ylabel, value):
            fig, ax = plt.subplots()
            plt.xlabel("Time Deployed (Months)")
            plt.ylabel(ylabel)
            plt.title(title)
            locator = ticker.MaxNLocator(nbins='auto', integer=False)
            ticks = locator.tick_values(0, value)
            top_tick = ticks[ticks > value][0]
            ax.set_xlim([0, duration_val])
            ax.set_ylim([0, top_tick])
            ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins='auto', integer=True))
            ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins='auto'))
            X = np.linspace(0, duration_val)
            Y = np.linspace(0, value)
            plt.plot(X, Y)
            plt.grid(True)
            plt.show()

        # ---- Generate Plots ----

        plot_with_top_limit("Charge vs Time Deployed", "Charge (Ah)", charge)
        plot_with_top_limit("Battery Weight vs Time Deployed", "Battery Weight (lbs)", weight)
        plot_with_top_limit("SD Card Size vs Time Deployed", "SD Card Size (GB)", sd_size)

In [175]:
# --------------------------
# ----- Show Tool UI ------
# --------------------------

output = Output()

interactive_plot = interactive_output(
    update_all_outputs,
    {
        'duration_val': duration,
        'inference_rate_val': inference_rate,
        'battery_type_val': battery_type
    }
)

display(ui, output)


Output()